<a href="https://colab.research.google.com/github/shernee/04_cmpe258/blob/master/FastAI_DataAugmentation_%5Bl%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### FastAI Data Augmentation Techniques using Pets dataset from fastai

In [ ]:
# Imports 

from fastai.vision.all import *

In [ ]:
# Download dataset

path = untar_data(URLs.PETS)

In [ ]:
# Get dataset into dataloader

dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()), get_items=get_image_files, get_y=parent_label, item_tfms=Resize(460), batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64)

In [ ]:
# Train baseline model 

model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.001289,0.000001,1.000000,02:50
1,0.004402,0.002363,0.999662,02:47


#### Progressive resizing: Gradually using larger and larger images to train.

In [ ]:
# Crop and normalize images

def get_dls(bs, size):
  dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])
  return dblock.dataloaders(path, bs=bs)

In [ ]:
# Image size = 128 (training on smaller images first)

dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(n_out=dls.c), loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.004412,0.000050,1.000000,02:11
1,0.003081,0.000112,1.000000,02:18


In [ ]:
# Image size = 224 (training on larger images later)

learn.dls = get_dls(64, 224)
learn.fine_tune(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.153712,318.044434,0.497970,02:47


epoch,train_loss,valid_loss,accuracy,time
0,0.003027,0.000494,0.999662,02:46
1,0.003077,0.000005,1.000000,02:46


With progressive resizing, the training and validation loss reduces drastically after 2 epochs as compared to the baseline model.

#### Test time augmentation - During inference or validation, creating multiple versions of each image, using data augmentation, and then taking the average or maximum of the predictions for each augmented version of the image.

In [ ]:
preds,targs = learn.tta()
accuracy(preds, targs).item()

1.0

#### Mixup - mixing up images using some linear combination of base image and target image 

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=MixUp())
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.248511,0.031284,0.996617,02:11
1,0.213328,0.017494,0.999662,02:10


In this case, augmenting data by mixing up made it difficult for the model to learn and after 2 iterations, the loss was higher and accuracy lower as compared to the base model. (this might improve if more number of epochs are tried, however, due to compute and time constraints, only 2 epochs were tried)